# Visualizando imágenes

Libreta para seguir el conjunto de imagenes que vayan saliendo para el área de estudio.

### Inicio

In [1]:
import os
os.getcwd()

'C:\\Users\\usuario\\Documents\\00-notebooks-2022'

In [1]:
# Earth Engine
import ee
ee.Initialize()

# geemap
import geemap
import geemap.colormaps as cmp # Paletas para visualización de imágenes (cm se confunde con matplotlib)

# Otras librerías
import pandas as pd
from ipywidgets import interact, fixed

# Archivos .py con funciones
from vis_tools import *

In [32]:
# Por cada cambio se necesita usar importlib.reload
import importlib # Para volver a importar el archivo funciones.py posterior a cada cambio
import sys
importlib.reload(sys.modules['vis_tools'])

<module 'vis_tools' from 'C:\\Users\\usuario\\Documents\\00-notebooks-2022\\vis_tools.py'>

### Assets y ROI

In [2]:
# Diccionario de visualización de imágenes Landsat 8 y 9 TOA
vis_rgb = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']} 

In [3]:
# Assets
asset_dept = ee.FeatureCollection('users/CesarVilca/departamentos')
asset_prov = ee.FeatureCollection('users/CesarVilca/provincias')
asset_dist = ee.FeatureCollection('users/CesarVilca/distritos')

# Filtrado
deptSelectedName = 'LAMBAYEQUE'
provSelectedName = 'FERREÑAFE' # 'FERREÑAFE' 'CHICLAYO'
distSelectedName = 'FERREÑAFE' # 'FERREÑAFE' 'CHONGOYAPE'

dept_fcol = asset_dept.filterMetadata('DEPARTAMEN', 'equals', deptSelectedName)
prov_fcol = asset_prov.filterMetadata('PROVINCIA', 'equals', provSelectedName)
dist_fcol = asset_dist.filterMetadata('DISTRITO', 'equals', distSelectedName)

# ROI Caja
roi = dist_fcol.geometry().bounds()

empty = ee.Image().paint(dist_fcol, 2,2)
empty2 = ee.Image().paint(roi, 2, 2)

In [6]:
# Visualización
Map = geemap.Map(layer_ctrl = True) 
Map.centerObject(dist_fcol, 9)

Map.addLayer(dept_fcol, {'color':'red'}, f'Departamento {deptSelectedName}')
Map.addLayer(prov_fcol, {'color':'green'}, f'Provincia {provSelectedName} ')
Map.addLayer(dist_fcol, {'color':'blue'}, f'Distrito {distSelectedName}')

# Bordes
Map.addLayer(empty , {}, 'Distrito Seleccionado')
Map.addLayer(empty2, {}, 'Limites')

Map

Map(center=[-6.6205221157277006, -79.79111975903714], controls=(WidgetControl(options=['position', 'transparen…

In [4]:
ruta_ferrenafe = r'files/Ferreñafe_areas.shp' # en Local
# ruta_ferrenafe = r'/content/Ferreñafe_areas.shp' # en Colab

predios_agricolas = geemap.shp_to_ee(ruta_ferrenafe)
predios_vis = ee.Image().paint(predios_agricolas, 1, 1) # Para visualización con geemap

predios_bound = predios_agricolas.geometry().bounds()
predios_bound_vis = ee.Image().paint(predios_bound, 1, 1) # Para visualización con geemap

type(predios_agricolas)

ee.featurecollection.FeatureCollection

### Landsat 8 y 9

In [5]:
# Filtrado
path = 10
row = 65 # 64

fecha = '2021-12'
meses = 6

In [6]:
tier = 'T1'
snippet_name = f"LANDSAT/LC08/C02/{tier}"

# Generando L9 raw

icol_l9 = (
    ee.ImageCollection(f'LANDSAT/LC09/C02/{tier}')
    .filterDate(fecha, ee.Date(fecha).advance(meses, 'month'))
    .filterMetadata('WRS_PATH', 'equals', path)
    .filterMetadata('WRS_ROW', 'equals', row)
)

reporteL9_raw = generar_reporte(icol_l9).sort_values('Fecha Corta')

# Generando L8 raw

icol_l8 = (
    ee.ImageCollection(f'LANDSAT/LC08/C02/{tier}')
    .filterDate(fecha, ee.Date(fecha).advance(meses, 'month'))
    .filterMetadata('WRS_PATH', 'equals', path)
    .filterMetadata('WRS_ROW', 'equals', row)
)

reporteL8_raw = generar_reporte(icol_l8)

# Concatenar DataFrames
df_l8_l9 = pd.concat([reporteL9_raw, reporteL8_raw])

# Ordenar fechas
df_l8_l9 = df_l8_l9.sort_values('Fecha Corta') # De mayor a menor usar: ,ascending=False

df_l8_l9 = df_l8_l9.reset_index(drop=True)
df_l8_l9

,ID GEE,Fecha Precisa,Fecha Corta,Año,Mes,Día,Día Juliano,CLOUD_COVER,SUN_ELEVATION
0,LANDSAT/LC08/C02/T1/LC08_010065_20211204,2021-12-04 10:28:39.679,2021-12-04,2021,12,4,338,41.76,61.231303
1,LANDSAT/LC09/C02/T1/LC09_010065_20211212,2021-12-12 10:28:36.901,2021-12-12,2021,12,12,346,46.70,60.112699
2,LANDSAT/LC08/C02/T1/LC08_010065_20211220,2021-12-20 10:28:37.250,2021-12-20,2021,12,20,354,67.62,59.196546
3,LANDSAT/LC09/C02/T1/LC09_010065_20211228,2021-12-28 10:28:35.288,2021-12-28,2021,12,28,362,83.58,58.451368
4,LANDSAT/LC08/C02/T1/LC08_010065_20220105,2022-01-05 10:28:32.585,2022-01-05,2022,1,5,5,42.07,57.964139
5,LANDSAT/LC09/C02/T1/LC09_010065_20220113,2022-01-13 10:28:31.598,2022-01-13,2022,1,13,13,32.94,57.699672
6,LANDSAT/LC08/C02/T1/LC08_010065_20220121,2022-01-21 10:28:28.991,2022-01-21,2022,1,21,21,37.22,57.678164
7,LANDSAT/LC09/C02/T1/LC09_010065_20220129,2022-01-29 10:28:34.396,2022-01-29,2022,1,29,29,21.12,57.843961
8,LANDSAT/LC08/C02/T1/LC08_010065_20220206,2022-02-06 10:28:25.404,2022-02-06,2022,2,6,37,52.05,58.152624
9,LANDSAT/LC09/C02/T1/LC09_010065_20220214,2022-02-14 10:28:30.097,2022-02-14,2022,2,14,45,88.04,58.557032


In [7]:
# Guardar dataframe
# df_l8_l9.to_csv('reporte-L8_L9.csv', index=False, encoding='utf-8-sig')

In [37]:
# Imagenes totales (Landsat 8 y 9)
icol_l9.merge(icol_l8).size().getInfo()

20

In [10]:
# Exportar csv
# df_l8_l9.to_csv('df_l8_l9.csv', index=False, encoding='utf-8-sig')

In [38]:
df_reporte = df_l8_l9
  
# Visualizar imágenes por mes
mes = df_reporte['Mes'].unique().tolist()
interact(ver_imgs_mensual, 
         mes=mes, 
         snippet_name =fixed(snippet_name), 
         df=fixed(df_reporte), 
         roi=fixed(predios_agricolas));

interactive(children=(Dropdown(description='mes', options=(12, 1, 2, 3, 4, 5), value=12), Output()), _dom_clas…

<function vis_tools.ver_imgs_mensual(mes, df, snippet_name, roi)>

### Sentinel-2

In [8]:
# Filtrado Sentinel-2
snippet_name_s2 = 'COPERNICUS/S2'
# snippet_name_s2 = 'COPERNICUS/S2_SR'

icol_s2_sr = (
    ee.ImageCollection(snippet_name_s2)
    .filterDate(fecha, ee.Date(fecha).advance(meses, 'month'))
    .filterBounds(predios_agricolas)
)

reporteS2 = generar_reporte(icol_s2_sr)
reporteS2

,ID GEE,Fecha Precisa,Fecha Corta,Año,Mes,Día,Día Juliano,MEAN SOLAR ZENITH ANGLE,CLOUDY PIXEL PERCENTAGE
0,COPERNICUS/S2/20211202T153621_20211202T154056_...,2021-12-02 10:45:07.038,2021-12-02,2021,12,2,336,25.107220,74.247100
1,COPERNICUS/S2/20211207T153619_20211207T153726_...,2021-12-07 10:45:02.453,2021-12-07,2021,12,7,341,25.872739,27.958400
2,COPERNICUS/S2/20211212T153621_20211212T154103_...,2021-12-12 10:45:06.775,2021-12-12,2021,12,12,346,26.540952,14.059200
3,COPERNICUS/S2/20211217T153609_20211217T153742_...,2021-12-17 10:45:01.362,2021-12-17,2021,12,17,351,27.156068,27.977200
4,COPERNICUS/S2/20211222T153621_20211222T154116_...,2021-12-22 10:45:08.640,2021-12-22,2021,12,22,356,27.636708,3.735200
5,COPERNICUS/S2/20211227T153619_20211227T153744_...,2021-12-27 10:45:03.076,2021-12-27,2021,12,27,361,28.058020,89.255500
6,COPERNICUS/S2/20220101T153621_20220101T154106_...,2022-01-01 10:45:09.178,2022-01-01,2022,1,1,1,28.337002,14.693500
7,COPERNICUS/S2/20220106T153619_20220106T153727_...,2022-01-06 10:45:03.572,2022-01-06,2022,1,6,6,28.547772,5.802700
8,COPERNICUS/S2/20220111T153621_20220111T154058_...,2022-01-11 10:45:08.654,2022-01-11,2022,1,11,11,28.615786,0.762000
9,COPERNICUS/S2/20220116T153619_20220116T153730_...,2022-01-16 10:45:03.128,2022-01-16,2022,1,16,16,28.618463,93.014300


In [9]:
# Exportar .csv
# reporteS2.to_csv('reporte-S2.csv', index=False, encoding='utf-8-sig')

In [29]:
df_reporte = reporteS2

# Ver imágenes por mes
mes = df_reporte['Mes'].unique().tolist()
interact(ver_imgs_mensual, 
         mes=mes, 
         snippet_name =fixed(snippet_name_s2), 
         df=fixed(df_reporte), 
         roi=fixed(predios_agricolas));

interactive(children=(Dropdown(description='mes', options=(12, 1, 2, 3, 4, 5), value=12), Output()), _dom_clas…

### Histórico de imágenes Landsat y Sentinel-2

**Gráficas interactivas con plotly**

Referencias:
- [Python Figure Reference: layout](https://plotly.com/python/reference/layout/)

In [10]:
import plotly.graph_objects as go

In [51]:
# Visualización con Plotly

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=reporteL8_raw['Fecha Precisa'], 
               y=reporteL8_raw['CLOUD_COVER'],
               name='LC08/C02/T1 ', 
               marker_color= 'rgba(0, 0, 0, 0.7)',
               mode='lines+markers',
               xhoverformat="%d-%m-%Y",
               hovertemplate='%{y}%',
               line = dict(width=2)
               )
    )

fig.add_trace(
    go.Scatter(x=reporteS2['Fecha Precisa'], 
               y=reporteS2['CLOUDY PIXEL PERCENTAGE'],
               name='Sentinel-2', 
               marker_color= 'rgba(0, 0, 0, 0.7)',
               mode='lines+markers',
               xhoverformat="%d-%m-%Y",
               hovertemplate='%{y:.2f}%',
               line = dict(width=2, dash='dot')
               )
    )

fig.add_trace(
    go.Scatter(x=reporteL9_raw['Fecha Precisa'], 
               y=reporteL9_raw['CLOUD_COVER'], 
               name='LC09/C02/T1', 
               marker_color= 'blue',
               mode='lines+markers', marker_size=7,
               xhoverformat="%d-%m-%Y",
               hovertemplate='%{y}%',
               line = dict(width=2)
              #  line = dict(width=2, dash='dot')
               )
    )

# fig.update_traces(mode = 'markers', marker_size=7)

# Título y hover
title_plotly = 'Imágenes LC09/C02/T1_TOA y LC08/C02/T1'

fig.update_layout(
    plot_bgcolor = "white",
    width = 800,
    height = 400,
    hovermode="x unified",
    title = title_plotly,
    yaxis_range = [0,100],
#     xaxis_range = [fecha, pd.to_datetime(fecha) + relativedelta(months=meses)],
#  https://stackoverflow.com/questions/4130922/how-to-increment-datetime-by-custom-months-in-python-without-using-library
    margin=dict(t=75, b=25, l=0, r=0),
    legend=dict(x=0.27, y=-0.10),
    legend_orientation='h',
    font_family='sans-serif'
)

# Nombres en ejes
fig.update_yaxes(title_text='Nubosidad (%)',
                 showline=False, linecolor='black', mirror=True,
                 gridcolor='#7f7f7f'
)

# Obtener ticks mensuales
fig.update_xaxes(
    showline=True, 
    linecolor='black', mirror=True,
    gridcolor='#7f7f7f',
    dtick='M1', tickformat='%m-%Y'
    )

fig.show()

**Exportar gráfica como archivo `.html`**

- [Plotly: Interactive HTML Export in Python](https://plotly.com/python/interactive-html-export/)

In [52]:
# Exportar grafica como archivo html
fig.write_html('plotly.html')